In [75]:
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from typing import List

class RepresentationModuleEncoder(nn.Module):
    def __init__(self,
                 snp_dim: int,
                 latent_dim: int,
                 hidden_dims: List = None,
                 **kwargs) -> None:
        super().__init__()

        self.latent_dim = latent_dim

        modules = []
        if hidden_dims is None:
            hidden_dims = [32, 64, 128, 256, 512]

        in_dim = snp_dim
        # Build Encoder
        for h_dim in hidden_dims:
            modules.append(
                nn.Sequential(
                    nn.Linear(in_dim, h_dim),
                    nn.ELU())
            )
            in_dim = h_dim

        modules.append(nn.Linear(hidden_dims[-1], latent_dim * 2))
        self.encoder = nn.Sequential(*modules)




    def encode(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        result = self.encoder(x)

        mu = result[:, :self.latent_dim]
        log_var = result[:, self.latent_dim:]

        return mu, log_var


    def reparameterize(self, mu: torch.Tensor, logvar: torch.Tensor) -> torch.Tensor:
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu


    def forward(self, x: torch.Tensor, **kwargs) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        return mu, log_var, z



class RepresentationModuleDecoder(nn.Module):
    def __init__(self,snp_dim: int, latent_dim, hidden_dims):
        super().__init__()

                # Build Decoder
        modules = []

        if hidden_dims is None:
            hidden_dims = [32, 64, 128, 256, 512]

        hidden_dims = hidden_dims[::-1]
        in_dim = latent_dim
        for h_dim in hidden_dims:
            modules.append(
                nn.Sequential(
                    nn.Linear(in_dim, h_dim),
                    nn.ELU())
            )
            in_dim = h_dim

        modules.append(nn.Linear(hidden_dims[-1], snp_dim))
        self.decoder = nn.Sequential(*modules)

    def decode(self, z: torch.Tensor) -> torch.Tensor:
        result = self.decoder(z)
        return result

    def forward(self, z: torch.Tensor):
        return self.decode(z)


In [76]:
class AssociationModuleGenerator(nn.Module):
    def __init__(self, input_dim: int, generator_hidden_dims: list[int], image_dim: int):
        super().__init__()
        # AssociationModule is similar to a GAN consisting of a generator and a discriminator
        # The generator generates from the latent space consisting of the output from the representation module concatenated with a demographic vector
        # It then outputs a fake image vector xmri and an attentive mask a
        # the discriminator takes the fake image vector and the real image vector and outputs a probability of the image being real

        # The generator is a simple feedforward network with the latent space concatenated with the demographic vector
        # The discriminator is a simple feedforward network with the image vector as input

        generator_modules = []
        if generator_hidden_dims is None:
            generator_hidden_dims = [32, 64, 128, 256, 512]

        current_dim = input_dim
        for h_dim in generator_hidden_dims:
            generator_modules.append(
                nn.Sequential(
                    nn.Linear(current_dim, h_dim),
                    nn.ELU())
            )
            current_dim = h_dim

        generator_modules.append(nn.Sequential(nn.Linear(generator_hidden_dims[-1], image_dim * 2), nn.Sigmoid()))

        self.generator = nn.Sequential(*generator_modules)


    def forward(self, x: torch.Tensor, demographic: torch.Tensor, **kwargs) -> tuple[torch.Tensor, torch.Tensor]:
        # concatenate the demographic vector with the latent space
        x = torch.cat((x, demographic), dim=1)
        generator_output = self.generator(x)
        # split the output into the fake image vector and the attentive mask by splitting the output in half
        fake_image = generator_output[:, :generator_output.shape[1] // 2]
        attentive_mask = generator_output[:, generator_output.shape[1] // 2:]
        return fake_image, attentive_mask

class AssociationModuleDiscriminator(nn.Module):
    def __init__(self, image_dim: int):
        super().__init__()
        self.discriminator = nn.Sequential(
            nn.Linear(image_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, x: torch.Tensor, **kwargs) -> torch.Tensor:
        return self.discriminator(x)

In [77]:
class DiagnosticianModule (nn.Module):
    def __init__(self, input_dim: int, reduction_dim: int, classification_targets: int):
        super().__init__()
        # perform regression and classification using two linear layers
        # after reducing dimensionality to
        self.dim_reduction = nn.Sequential(
            nn.Linear(input_dim, reduction_dim),
            nn.ELU()
        )

        self.classifier = nn.Linear(reduction_dim, classification_targets)
        self.regressor = nn.Linear(reduction_dim, 1)

    def forward(self, x: torch.Tensor, apply_logistic_activation: bool, **kwargs) -> tuple[torch.Tensor, torch.Tensor]:
        reduced_dims = self.dim_reduction(x)
        classification_output = self.classifier(reduced_dims)
        regression_output = self.regressor(reduced_dims)
        if apply_logistic_activation:
            y_hat = nn.functional.softmax(classification_output)
            s_hat = nn.functional.sigmoid(regression_output)
            return y_hat, s_hat
        return classification_output, regression_output

In [78]:
class GenerativeDiscriminativeModel(nn.Module):
    def __init__(self, snp_dims: int, mri_dims: int, demographic_dims, classification_dims: int):
        super().__init__()
        self.encoder = RepresentationModuleEncoder(snp_dims, 50, [500])
        self.decoder = RepresentationModuleDecoder(snp_dims, 50, [500])

        self.generator = AssociationModuleGenerator(50 + demographic_dims, [100], mri_dims)
        self.discriminator = AssociationModuleDiscriminator(mri_dims)

        self.diagnostician = DiagnosticianModule(mri_dims, 25, classification_dims)

    def forward(self, snp_features: torch.Tensor, mri_features: torch.Tensor, demographic_features: torch.Tensor):
        mu, log_var, z = self.encoder(snp_features)
        snp_reconstruction = self.decoder(z)

        # concatenate z and demographic features
        xmri_fake, attention_mask = self.generator(z, demographic_features)
        xmri_real = mri_features
        discriminator_output_fake = self.discriminator(xmri_fake)
        discriminator_output_real = self.discriminator(xmri_real)

        # hadamard product between the attention mask and the real image
        attended_mri_features = attention_mask * xmri_real

        y_logits, mmsr_regression = self.diagnostician(attended_mri_features, False)

        return snp_reconstruction, mu, log_var, discriminator_output_fake, discriminator_output_real, y_logits, mmsr_regression

In [79]:
from torch.utils.data import Dataset

class AdniDataset(Dataset):
    def __init__(self, snp_data: np.ndarray, mri_data: np.ndarray, demographic_data: np.ndarray, diagnosis_data: np.ndarray):
        self.raw_snp_data = np.copy(snp_data)
        self.mri_data = torch.from_numpy(np.copy(mri_data)).to(dtype=torch.float32)
        self.demographic_data = torch.from_numpy(np.copy(demographic_data)).to(dtype=torch.float32)
        diagnosis_data = np.copy(diagnosis_data)
        self.mmse_data = torch.from_numpy(diagnosis_data[:, 1]).to(dtype=torch.float32)
        self.diagnosis_data = torch.from_numpy(diagnosis_data[:, 0]).to(dtype=torch.float32)
        self.snp_data = torch.zeros((self.raw_snp_data.shape[0], self.raw_snp_data.shape[1]))


    def normalize(self, normalization_matrix: np.ndarray | None = None) -> tuple[np.ndarray, np.ndarray]:
        # we have to normalize snp data by computing a normalization matrix. We want as rows all possible values and as columns probability of that value in the dataset
        # we then use this matrix to normalize the snp data

        # get all unique values in the snp data
        if normalization_matrix is None:
            unique_values = np.unique(self.raw_snp_data)
            normalization_matrix = np.zeros((len(unique_values), self.raw_snp_data.shape[1]))
            for i, value in enumerate(unique_values):
                normalization_matrix[i] = (self.raw_snp_data == value).sum(axis=0) / self.raw_snp_data.shape[0]

        normalized_snp_data = np.zeros((self.raw_snp_data.shape[0], self.raw_snp_data.shape[1]))
        # we now have a matrix where each row is a unique value and each column is the probability of that value in the dataset
        # we can now normalize the snp data by replacing each value with the corresponding row in the normalization matrix
        for i in range(self.raw_snp_data.shape[0]):
            for j in range(self.raw_snp_data.shape[1]):
                normalized_snp_data[i, j] = normalization_matrix[self.raw_snp_data[i, j], j]

        self.snp_data = torch.from_numpy(normalized_snp_data).to(dtype=torch.float32)
        return normalized_snp_data, normalization_matrix

    def __len__(self):
        return len(self.snp_data)

    def __getitem__(self, idx):
        return self.snp_data[idx], self.mri_data[idx], self.demographic_data[idx], self.diagnosis_data[idx], self.mmse_data[idx]


In [80]:
import numpy as  np
#dataset_base_path = "/media/jfallmann/T9/University/master_thesis/dataset"
dataset_base_path = "/Volumes/T9/University/master_thesis/dataset"

mri_raw_path = f"{dataset_base_path}/mri/raw"
mri_base_path = f"{dataset_base_path}/mri"
snp_raw_path = f"{dataset_base_path}/snp/raw"
mri_bids_path = f"{dataset_base_path}/mri/bids"
mri_fastsurfer_out = f"{dataset_base_path}/mri/processed"
tables_path = f"{dataset_base_path}/tables"

In [81]:
# import mri data
mri_data = np.load(f"{mri_base_path}/processed_volumes.npy")
# import snp data
snp_data = np.load(f"{dataset_base_path}/snp/processed/genomes.npy")
# import demographic data
demographic_data = np.load(f"{dataset_base_path}/tables/demographic_data.npy")
# import diagnosis data
diagnosis_data = np.load(f"{dataset_base_path}/tables/diagnosis_data.npy")

zero_rows = np.where(~mri_data.any(axis=1))[0]
mri_data = np.delete(mri_data, zero_rows, axis=0)
snp_data = np.delete(snp_data, zero_rows, axis=0)
demographic_data = np.delete(demographic_data, zero_rows, axis=0)
diagnosis_data = np.delete(diagnosis_data, zero_rows, axis=0)

In [82]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

def get_dataloader(classification_mode: str, batch_size, split, full_snp_data,full_mri_data, full_demographic_data, full_diagnosis_data):
    Y = diagnosis_data[:, 0]
    rows = None
    if classification_mode == "cn/ad":
        rows = np.where((Y == 1) | (Y == 3))

    c_snp_data = full_snp_data[rows]
    c_mri_data = full_mri_data[rows]
    c_demographic_data = full_demographic_data[rows]
    c_diagnosis_data = full_diagnosis_data[rows]

    diagnosis = c_diagnosis_data[:, 0]
    diagnosis = (diagnosis - np.min(diagnosis)) / (np.max(diagnosis) - np.min(diagnosis))
    c_diagnosis_data[:, 0] = diagnosis

    snp_train, snp_test, mri_train, mri_test, demographic_train, demographic_test, diagnosis_train, diagnosis_test = train_test_split(c_snp_data, c_mri_data, c_demographic_data, c_diagnosis_data, test_size=split, random_state=42)

    train_dataset = AdniDataset(snp_train, mri_train, demographic_train, diagnosis_train)
    _, normalization_matrix = train_dataset.normalize()

    test_dataset = AdniDataset(snp_test, mri_test, demographic_test, diagnosis_test)
    _, _ = test_dataset.normalize(normalization_matrix)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [83]:
def elbo(reconstruction, input, mu, log_var, kld_weight) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    recons_loss =F.mse_loss(reconstruction, input)
    kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp(), dim = 1), dim = 0)

    loss = recons_loss + kld_weight * kld_loss
    return loss,recons_loss.detach(),-kld_loss.detach()

In [89]:
from tqdm import tqdm
from torch.optim.lr_scheduler import ExponentialLR

def train(model, device,train_loader, optimizer, num_epochs):
    model = model.to(device)
    model.train()
    train_losses = []
    generator_loss_function = nn.MSELoss().to(device)
    discriminator_loss_function = nn.MSELoss().to(device)
    classification_loss_function = nn.BCEWithLogitsLoss().to(device)
    regression_loss_function = nn.MSELoss().to(device)
    scheduler = ExponentialLR(optimizer, gamma=0.95)

    for epoch in range(num_epochs):
        total_loss = 0
        elbo_losses = []
        generator_losses = []
        discriminator_losses = []
        classification_losses = []
        regression_losses = []

        for snp, mri, demographic, diagnosis, mmse in train_loader:
            optimizer.zero_grad()
            snp = snp.to(device)
            mri = mri.to(device)
            demographic = demographic.to(device)
            diagnosis = diagnosis.unsqueeze(1).to(device)
            mmse = diagnosis.to(device)
            snp_reconstruction, mu, log_var, discriminator_output_fake, discriminator_output_real, y_logits, mmsr_regression = model(snp, mri, demographic)

            # calculate losses
            elbo_loss, recon_loss, kld_loss = elbo(snp_reconstruction, snp, mu, log_var, 0.1)
            generator_loss = generator_loss_function(discriminator_output_fake, torch.ones_like(discriminator_output_fake))
            discriminator_loss = discriminator_loss_function(discriminator_output_real, torch.ones_like(discriminator_output_real)) + discriminator_loss_function(discriminator_output_fake, torch.zeros_like(discriminator_output_fake))
            classification_loss = classification_loss_function(y_logits, diagnosis)
            regression_loss = regression_loss_function(mmsr_regression, mmse)

            loss = (0.7*elbo_loss + 0.5*generator_loss + discriminator_loss + classification_loss + 0.7*regression_loss)/ 3.9

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            elbo_losses.append(elbo_loss.item())
            generator_losses.append(generator_loss.item())
            discriminator_losses.append(discriminator_loss.item())
            classification_losses.append(classification_loss.item())
            regression_losses.append(regression_loss.item())
        train_losses.append(total_loss)

        if epoch % 10 == 0:
            scheduler.step()
            print(f"Epoch {epoch} loss: {total_loss}. Elbo loss: {np.mean(elbo_losses)}. Generator loss: {np.mean(generator_losses)}. Discriminator loss: {np.mean(discriminator_losses)}. Classification loss: {np.mean(classification_losses)}. Regression loss: {np.mean(regression_losses)}")
    return train_losses

In [90]:
# create model
diagnosis = [1, 3]
model = GenerativeDiscriminativeModel(snp_data.shape[1], mri_data.shape[1], demographic_data.shape[1], 1)
num_epochs = 2000
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
batch_size = 5
device = 'cpu'

train_loader, test_loader = get_dataloader("cn/ad", batch_size, 0.2, snp_data, mri_data, demographic_data, diagnosis_data)

train_losses = train(model,device, train_loader, optimizer, num_epochs)

Epoch 0 loss: 14.385527670383453. Elbo loss: 0.5104268496074984. Generator loss: 0.4237694634545234. Discriminator loss: 0.39360825188698306. Classification loss: 0.6943013937242569. Regression loss: 0.21814157259500316
Epoch 10 loss: 9.742264851927757. Elbo loss: 0.041055087240472916. Generator loss: 0.44343765320316436. Discriminator loss: 0.28914570327728023. Classification loss: 0.5534464466956354. Regression loss: 0.18941468721435917
Epoch 20 loss: 9.383879274129868. Elbo loss: 0.084534632342477. Generator loss: 0.44414996620147457. Discriminator loss: 0.229658686826306. Classification loss: 0.5422667245711049. Regression loss: 0.18196848876053287
Epoch 30 loss: 9.167731329798698. Elbo loss: 0.16243517350765965. Generator loss: 0.44374262517498386. Discriminator loss: 0.19313270766888896. Classification loss: 0.5127860355761743. Regression loss: 0.1598072913625548
Epoch 40 loss: 8.458666801452637. Elbo loss: 0.177872953636031. Generator loss: 0.4453290306752728. Discriminator loss